In [1]:
import os
import pandas as pd
import numpy as np
from mol_lib import xtb_isomers, FromXtbToSpinM, FromFormulaToAtomsList, FromFormulaToElectronCore, FromFormulaToElectron, FromNetworkDataFrameToNetworkClassList
import matplotlib.pyplot as plt

with open("/Users/tinaccil/Documents/GitHub/woon_query/data_analysis/dataframe/molecules_info.csv","r") as data:
	df_mol = pd.read_csv(data, delimiter = '\t')
with open("/Users/tinaccil/Documents/GitHub/woon_query/data_analysis/dataframe/network.csv","r") as data:
	df_net = pd.read_csv(data, delimiter = '\t')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_net)

RDKit WARNING: [18:02:21] Enabling RDKit 2019.09.3 jupyter extensions


,R1,R2,R3,P1,P2,P3,P4,P5,ID_kida,Reaction_Energy(kJ/mol)_Woon,Reaction_Energy(kJ/mol)_xtb,Reaction_Image
0,C2_0,H1C1N1_0,NaN,H1_0,C3N1_6,NaN,NaN,NaN,0,5.860120,149.312272,reaction_0
1,C2_0,H1C1N1_1,NaN,H1_0,C3N1_6,NaN,NaN,NaN,0,-56.251375,65.540833,reaction_1
2,O1_0,H2C1_0,NaN,H1_0,H1C1O1_0,NaN,NaN,NaN,1,-362.802332,-660.931495,reaction_2
3,O1_0,H2C1_0,NaN,C1O1_0,H2_0,NaN,NaN,NaN,2,-737.881509,-965.221492,reaction_3
4,O1_0,H2C1_0,NaN,C1O1_0,H1_0,H1_0,NaN,NaN,3,-283.627701,-451.358396,reaction_4
5,O1_0,H1C1O1_0,NaN,H1_0,C1O2_1,NaN,NaN,NaN,4,-457.580319,-885.735974,reaction_5
6,O1_0,H1C1O1_0,NaN,C1O1_0,H1O1_0,NaN,NaN,NaN,5,-371.429730,-487.421353,reaction_6
7,O1_0,H1C1N1_0,NaN,C1O1_0,H1N1_0,NaN,NaN,NaN,6,-136.100759,-138.480647,reaction_7
8,O1_0,H1C1N1_1,NaN,C1O1_0,H1N1_0,NaN,NaN,NaN,6,-198.212253,-222.252086,reaction_8
9,O1_0,H1C1N1_0,NaN,C1N1_0,H1O1_0,NaN,NaN,NaN,7,99.629914,-133.536547,reaction_9


In [ ]:
df_net['diff'] = df_net['Reaction_Energy(kJ/mol)_xtb'] - df_net['Reaction_Energy(kJ/mol)_Woon']
df_net.plot(x='ID_kida', y='diff', color='Red', kind='scatter',figsize=(20,10))
#df_net['diff'].diff().hist(bins=30)
tmp_df_net = df_net[np.abs(df_net['diff']) < 200]
#tmp_df_net = df_net.copy()

In [ ]:
f1 = tmp_df_net['R1'].value_counts().tolist()
n1 = tmp_df_net['R1'].value_counts().index.tolist()
f2 = tmp_df_net['R2'].value_counts().tolist()
n2 = tmp_df_net['R2'].value_counts().index.tolist()
f3 = tmp_df_net['P1'].value_counts().tolist()
n3 = tmp_df_net['P1'].value_counts().index.tolist()
f4 = tmp_df_net['P2'].value_counts().tolist()
n4 = tmp_df_net['P2'].value_counts().index.tolist()
f5 = tmp_df_net['P3'].value_counts().tolist()
n5 = tmp_df_net['P3'].value_counts().index.tolist()

df_mol_freq = df_mol.copy()
df_mol_freq = df_mol_freq.drop(columns={'Na','Mass','state_woon','multiplicity_woon','energy(kJ/mol)_woon','spin_input_xtb','multiplicity_xtb','energy(kJ/mol)_xtb','pwd_xyz_woon','pwd_xyz_xtb'})

df_mol_freq['f_R1'] = 0
df_mol_freq['f_R2'] = 0
df_mol_freq['f_P1'] = 0
df_mol_freq['f_P2'] = 0
df_mol_freq['f_P3'] = 0

for i,item in enumerate(f1):
    df_mol_freq.loc[df_mol_freq['ID'] == n1[i], 'f_R1'] = f1[i]
for i,item in enumerate(f2):
    df_mol_freq.loc[df_mol_freq['ID'] == n2[i], 'f_R2'] = f2[i]
for i,item in enumerate(f3):
    df_mol_freq.loc[df_mol_freq['ID'] == n3[i], 'f_P1'] = f3[i]
for i,item in enumerate(f4):
    df_mol_freq.loc[df_mol_freq['ID'] == n4[i], 'f_P2'] = f4[i]
for i,item in enumerate(f5):
    df_mol_freq.loc[df_mol_freq['ID'] == n5[i], 'f_P3'] = f5[i]

df_mol_freq['f_RT'] = df_mol_freq['f_R1'] + df_mol_freq['f_R2']
df_mol_freq['f_PT'] = df_mol_freq['f_P1'] + df_mol_freq['f_P2'] + df_mol_freq['f_P3']

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df_mol_freq)

df_mol_freq.plot(x="name", y=["f_RT", "f_PT"], kind="bar",legend=True, figsize=(20,10))

In [ ]:
net = FromNetworkDataFrameToNetworkClassList(tmp_df_net)
for i,items in enumerate(net):
    for j,item in enumerate(net[i].reactants):
        
    for j,item in enumerate(net[i].products):
			

In [ ]:
FromFormulaToElectronCore
df_mol['core_el'] = df_mol.apply(lambda row: FromFormulaToElectronCore(row['formula']), axis=1)
df_mol['electron'] = df_mol.apply(lambda row: FromFormulaToElectron(row['formula']), axis=1)
df_mol.sort_values(by=['electron',"Mass"], inplace=True)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df_mol)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(X, a, b, c):
    x,y  = X
    return a * x + b * y + c

name = df_mol['name'].tolist()
e_xtb = df_mol['energy(kJ/mol)_xtb'].tolist()
e_w = df_mol['energy(kJ/mol)_woon'].tolist()
e_ce = df_mol['core_el'].tolist()
e_e = df_mol['electron'].tolist()


params, params_covariance = curve_fit(func, (e_xtb,e_e), e_w)
print(params)

e_xtb_n = []
for i,item in enumerate(e_xtb):
    e_xtb_n.append(func((e_xtb[i],e_e[i]), params[0], params[1], params[2]))

plt.figure(figsize=(20, 10))
plt.scatter(name, e_w, label='woon', color='Red')
plt.scatter(name, e_xtb_n, label='f(xtb,ncore)', color='Blue')
plt.legend(loc='best')
plt.xticks(rotation=90)
#plt.ylim(1.45e-10,2.01e-10)
plt.show()   

In [ ]:
ax = df_mol.plot(x='name', y='energy(kJ/mol)_woon', kind='scatter', color='Blue', label='Woon')
df_mol.plot(x='name', y='energy(kJ/mol)_xtb', color='Red', kind='scatter',ax=ax, label='xTB', figsize=(20,10))
plt.xticks(rotation=90)

In [ ]:
#df_mol.plot(x='energy(kJ/mol)_woon', y='energy(kJ/mol)_xtb', color='Red', kind='scatter', figsize=(20,10))

fig, ax = plt.subplots(figsize=(30, 60))
ax.scatter(e_w, e_xtb)
for i, txt in enumerate(name):
    ax.annotate(txt, (e_w[i], e_xtb[i]))

In [ ]:
def norm(val,val_n):
    tmp = (val - val_n) / abs(val_n) 
    return(tmp)

max_val_x = df_mol['energy(kJ/mol)_xtb'].min()
df_mol['n_x'] = df_mol.apply(lambda row: norm(row['energy(kJ/mol)_xtb'],max_val_x), axis=1)

max_val_w = df_mol['energy(kJ/mol)_woon'].min()
df_mol['n_w'] = df_mol.apply(lambda row: norm(row['energy(kJ/mol)_woon'],max_val_w), axis=1)

ax = df_mol.plot(x='name', y='n_w', kind='scatter', color='Blue', label='Woon')
df_mol.plot(x='name', y='n_x', color='Red', kind='scatter',ax=ax, label='xTB', figsize=(20,10))
plt.xticks(rotation=90)


In [ ]:
'''
tmp_w = df_mol['multiplicity_woon'].tolist()
tmp_x = df_mol['multiplicity_xtb'].tolist()

tmp_b = []
for i,item in enumerate(tmp_w):
	if (int(tmp_w[i]) - int(tmp_x[i])) == 0:
		tmp_b.append(True)
	else:
		tmp_b.append(False)

df_mol['bool'] = tmp_b
df_mol_f = df_mol[df_mol['bool'] == False]
df_mol = df_mol.drop(["bool"], axis=1)
'''